## Prepare and read data

In [1]:
import gc
import itertools
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [4]:
def feature_cache(reset=False):
    def _feature_cache(func):
        def wrapper(train_df, test_df, *args):
            func_name = func.__name__
            train_feat_path = Path("../feature") / f"train_{func_name}.pkl"
            test_feat_path = Path("../feature") / f"test_{func_name}.pkl"
            # if feature exists, load feature
            if train_feat_path.exists() and test_feat_path.exists() and not reset:
                train_feats = pd.read_pickle(train_feat_path).reset_index(drop=True)
                test_feats = pd.read_pickle(test_feat_path).reset_index(drop=True)
                train_df = pd.concat([train_df, train_feats], axis=1)
                test_df = pd.concat([test_df, test_feats], axis=1)
            # if not exists, make feature and save as pickle
            else:
                before_cols = train_df.columns.tolist()
                train_df, test_df = func(train_df, test_df, *args)
                after_cols = train_df.columns.tolist()
                new_cols = [c for c in after_cols if c not in before_cols]
                train_feats = train_df[new_cols]
                test_feats = test_df[new_cols]
                train_feats.to_pickle(train_feat_path)
                test_feats.to_pickle(test_feat_path)            
            return train_df, test_df
        return wrapper

    return _feature_cache

## Feature Engineering

In [5]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [6]:
original_cols = train_df.columns.tolist()

In [7]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [8]:
nume_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
    if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
]

In [9]:
cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

In [10]:
len(original_cols), len(cat_cols), len(nume_cols), len(multi_cat_cols)

(128, 65, 40, 21)

### Multi-category encoding

In [11]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
    all_feat_df = pd.concat([train_feat_df, test_feat_df], axis=0, ignore_index=True)
    train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)
    # SVD
    svd = TruncatedSVD(n_components=2)
    all_svd_feats = pd.DataFrame(svd.fit_transform(all_feat_df), columns=[f"svd_{c}_{ix}" for ix in range(2)])
    train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)

### Simple-category encoding

In [12]:
for c in cat_cols:
    train_df[c], uniques = pd.factorize(train_df[c], sort=True)
    test_df[c] = uniques.get_indexer(test_df[c])

### Reduce data memory

In [13]:
for c in nume_cols:
    if train_df[c].nunique() > 1000:
        train_df[c] = train_df[c].astype(np.float32)
        test_df[c] = test_df[c].astype(np.float32)
    else:
        train_df[c] = train_df[c].astype(np.float16)
        test_df[c] = test_df[c].astype(np.float16)

In [14]:
for c in cat_cols:
    if train_df[c].max() > 32767:
        train_df[c] = train_df[c].astype(np.int32)
        test_df[c] = test_df[c].astype(np.int32)
    else:
        train_df[c] = train_df[c].astype(np.int16)
        test_df[c] = test_df[c].astype(np.int16)            

In [15]:
train_df.shape, test_df.shape

((33857, 507), (11259, 506))

### Change Category Name

In [16]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Aggregation

In [17]:
def get_importance_cols(use_num=50):
    # basicなモデルのimportanceを読み込み
    importance_df = pd.read_csv("../data/importance/003_importance.csv")
    imp_feats = importance_df["feature"].iloc[:use_num].tolist()
    return imp_feats

In [18]:
imp_feats = get_importance_cols(use_num=50)
imp_cat_cols = [c for c in imp_feats if c in cat_cols or c[:4] == "ohe_"]
imp_nume_cols = [c for c in imp_feats if c in nume_cols or c[:4] == "ohe_"]

In [19]:
len(imp_cat_cols), len(imp_nume_cols)

(32, 15)

In [20]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [21]:
for cat_col in tqdm(imp_cat_cols):
    for nume_col in imp_nume_cols:
        # one-hotは同じカテゴリの場合がある
        if cat_col == nume_col:
            continue
        all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
        all_df[f"agg_std_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
        all_df[f"agg_max_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
        all_df[f"agg_min_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
        all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
        all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
del all_df
gc.collect()

58

In [22]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [23]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [24]:
@feature_cache(reset=False)
def target_encoding(train_df, test_df):
    te_cols = [c for c in train_df.columns if c in cat_cols]
    for c in tqdm(te_cols):
        new_col = "te_" + c
        train_df[new_col] = 0
        test_df[new_col] = 0
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
    return train_df, test_df

In [25]:
train_df, test_df = target_encoding(train_df, test_df)

In [26]:
@feature_cache(reset=False)
def multiple_target_encoding(train_df, test_df):
     # multiple target encoding
    multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
    imp_feats = get_importance_cols(use_num=30)
    multi_te_cols = [c for c in multi_te_cols if c in imp_feats]
    combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))   

    for col_a, col_b in tqdm(combi_multi_te_cols):
        new_col = "te_" + col_a + "__" + col_b
        train_df[new_col] = 0
        test_df[new_col] = 0
        train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
        test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
        del train_df["tmp"], test_df["tmp"]
        gc.collect()
    return train_df, test_df

In [27]:
train_df, test_df = multiple_target_encoding(train_df, test_df)

In [28]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

3573


## Training

In [29]:
lgb_params = {
            'objective': 'tweedie',
            "tweedie_variance_power": 1.05,
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 64,
            'min_data_in_leaf': 80, 
            'max_depth': 4,
            "bagging_freq": 5,
            "bagging_fraction": 0.8,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.1,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [30]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 20960.6	valid_1's rmse: 21690.4
[200]	training's rmse: 19771	valid_1's rmse: 21074.8
[300]	training's rmse: 19137.6	valid_1's rmse: 20841.8
[400]	training's rmse: 18665.4	valid_1's rmse: 20717.9
[500]	training's rmse: 18240.5	valid_1's rmse: 20630.1
[600]	training's rmse: 17863.3	valid_1's rmse: 20597.4
[700]	training's rmse: 17545	valid_1's rmse: 20563.8
[800]	training's rmse: 17199.6	valid_1's rmse: 20525.9
[900]	training's rmse: 16886.8	valid_1's rmse: 20508
[1000]	training's rmse: 16600.4	valid_1's rmse: 20479.4
[1100]	training's rmse: 16340.1	valid_1's rmse: 20460.6
[1200]	training's rmse: 16071.5	valid_1's rmse: 20450.3
[1300]	training's rmse: 15830.6	valid_1's rmse: 20433.2
[1400]	training's rmse: 15590.4	valid_1's rmse: 20406.2
[1500]	training's rmse: 15322.9	valid_1's rmse: 20408.6
Early stopping, best iteration is:
[1402]	training's rmse: 15584.9	valid_1's rmse: 20403.6
Fold 2
Training

In [31]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20233.230939728062

In [32]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:300]

,feature,gain
0,te_Country__YearsCodingProf,3.540792e+08
1,te_Country__ohe_DevType_Student,3.509223e+08
2,te_Country__ohe_LanguageWorkedWith_PHP,2.218623e+08
3,te_Country__Age,2.058722e+08
4,te_Country__Dependents,1.858720e+08
5,te_Country__Employment,1.274744e+08
6,te_YearsCodingProf__MilitaryUS,1.150568e+08
7,te_Country__YearsCoding,1.134862e+08
8,te_YearsCodingProf__CurrencySymbol,1.030893e+08
9,te_YearsCodingProf__SalaryType,8.001464e+07


## Prediction

In [33]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [34]:
test_pred

array([ 69493.26514271,  93058.78859525,  84463.13183881, ...,
        87815.84352235,  83725.53336848, 120273.10786866])

In [35]:
sub_df = pd.read_csv("../input/submit.csv")

In [36]:
sub_df["Salary"] = test_pred

In [37]:
# sub_df.to_csv("../predict/018_poisson_20206.csv", index=False)